In [1]:
import numpy as np
import networkx as nx

In [2]:
data = []
with open('chowliu-input.txt') as f:
    for line in f:
        data.append(line.split())

data = [[int(i) for i in j] for j in data]
data = np.array(data)

In [3]:
name = []
with open('names.txt') as f:
    for line in f:
        name.append(line.split())

name = [i[0] for i in name]

In [6]:
class Chow_liu(object):
    def __init__(self, data, name):
        self.data = data
        self.name = name
    
    def compute_marginal_i(self):
        self.marg_i_1 = np.sum(self.data,axis=0)/len(self.data)
        return
        
    def compute_MI_i_j(self):
        self.compute_marginal_i()
        self.marg_i_0 = 1 - self.marg_i_1
        MI_table = np.zeros([len(self.name), len(self.name)])
        marg_i_j = np.zeros([2,2])
        #compute marginal_i_j and update mutual information table at the same time
        for i in range(self.data.shape[1]):
            for j in range(i):
                ji = list(zip(self.data[:,j], self.data[:,i]))
                marg_i_j[0,0] = ji.count((0,0))/self.data.shape[0]
                if marg_i_j[0,0] == 0:
                    marg_i_j[0,0] = self.marg_i_0[j]*self.marg_i_0[i]
                marg_i_j[0,1] = ji.count((0,1))/self.data.shape[0]
                if marg_i_j[0,1] == 0:
                    marg_i_j[0,1] = self.marg_i_0[j]*self.marg_i_1[i]
                marg_i_j[1,0] = ji.count((1,0))/self.data.shape[0]
                if marg_i_j[1,0] == 0:
                    marg_i_j[1,0] = self.marg_i_1[j]*self.marg_i_0[i]
                marg_i_j[1,1] = ji.count((1,1))/self.data.shape[0]
                if marg_i_j[1,1] == 0:
                    marg_i_j[1,1] = self.marg_i_1[j]*self.marg_i_1[i]
                MI_table[j,i] = marg_i_j[0,0] * np.log(marg_i_j[0,0]/(self.marg_i_0[j]*self.marg_i_0[i])) + \
                marg_i_j[0,1] * np.log(marg_i_j[0,1]/(self.marg_i_0[j]*self.marg_i_1[i])) + \
                marg_i_j[1,0] * np.log(marg_i_j[1,0]/(self.marg_i_1[j]*self.marg_i_0[i])) + \
                marg_i_j[1,1] * np.log(marg_i_j[1,1]/(self.marg_i_1[j]*self.marg_i_1[i]))
        self.MI_table = MI_table
        return
        
    def build_tree(self):
        self.compute_MI_i_j()
        g = nx.Graph()
        for i in range(self.MI_table.shape[1]):
            g.add_node(i)
            for j in range(i):
                g.add_edge(j, i, weight=-self.MI_table[j,i])
        g_mst = nx.minimum_spanning_tree(g)
        
        #compute edge potentials of g_mst
        edge_pot_ls = []
        for i in g_mst.edges:
            pot_x_y = np.zeros([2,2])
            x,y = i[0],i[1]
            xy = list(zip(data[:,x], data[:,y]))
            for ind in [(0,0),(0,1),(1,0),(1,1)]:
                if ind == (0,0):
                    pot_x_y[ind] = (xy.count(ind)/self.data.shape[0])/(self.marg_i_0[x]*self.marg_i_0[y])
                elif ind == (0,1):
                    pot_x_y[ind] = (xy.count(ind)/self.data.shape[0])/(self.marg_i_0[x]*self.marg_i_1[y])
                elif ind == (1,0):
                    pot_x_y[ind] = (xy.count(ind)/self.data.shape[0])/(self.marg_i_1[x]*self.marg_i_0[y])
                else:
                    pot_x_y[ind] = (xy.count(ind)/self.data.shape[0])/(self.marg_i_1[x]*self.marg_i_1[y])
            edge_pot_ls.append(pot_x_y)
            
        
        #shown as UAI format
        print('MARKOV')
        print(len(g_mst.nodes))
        #print("CARDINAILITES")
        print(len(g_mst.edges))
        for i in g_mst.edges:
            print(len(i), i[0], i[1])
        print("")
        print("Node Potentials")
        for i in g_mst.nodes:
            print(2)
            print(self.marg_i_0[i], self.marg_i_1[i])
            print("")
        print("")
        print("Edge Potentials")
        for i in edge_pot_ls:
            print(4)
            print(i[0][0], i[0][1])
            print(i[1][0], i[1][1])
            print("")
                    
        return g_mst

In [8]:
CL = Chow_liu(data, name)
g_mst = CL.build_tree()

MARKOV
111
CARDINAILITES
110
2 0 72
2 0 54
2 1 46
2 1 86
2 1 45
2 2 20
2 3 85
2 4 20
2 5 85
2 6 40
2 7 85
2 8 108
2 8 63
2 8 35
2 9 66
2 10 78
2 11 13
2 12 13
2 13 46
2 14 85
2 14 15
2 14 101
2 16 65
2 16 22
2 17 18
2 17 80
2 19 82
2 20 46
2 20 72
2 20 89
2 20 100
2 20 39
2 20 104
2 21 72
2 22 32
2 22 41
2 23 47
2 24 72
2 24 25
2 24 105
2 24 97
2 26 95
2 26 46
2 26 36
2 27 46
2 28 36
2 29 85
2 30 85
2 31 85
2 32 84
2 33 93
2 33 70
2 34 108
2 36 77
2 37 65
2 38 93
2 40 110
2 40 52
2 40 69
2 42 108
2 43 102
2 44 85
2 46 108
2 46 85
2 46 102
2 46 51
2 46 75
2 46 106
2 46 78
2 46 96
2 46 109
2 46 71
2 46 92
2 46 90
2 47 106
2 48 95
2 49 85
2 50 102
2 50 60
2 50 53
2 52 87
2 55 93
2 56 108
2 57 85
2 58 85
2 58 110
2 59 93
2 61 88
2 62 108
2 64 68
2 64 102
2 66 83
2 67 72
2 72 94
2 72 83
2 72 103
2 73 74
2 74 108
2 76 78
2 79 108
2 80 85
2 80 107
2 81 85
2 82 85
2 84 93
2 84 99
2 84 108
2 84 98
2 85 88
2 90 91

Node Potentials
2
0.9917563544767575 0.0082436455232425

2
0.96702541790703 0.032